In [58]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import math
import scipy.integrate as si
import scipy.optimize as so

In [59]:
# PARAMETERS

incremental_Efrac = 7.8  # incremental E_frac value of each successive encounter. Choose from one of the following values: 0.1, 7.8
save_to_disk = False # If set to True, figure will be saved to disk. Set to false if you don't want figure to be saved to disk.

In [60]:
def second_derivative_parametric_input(x, y, t):
    # Input should be 'x' in terms 't', and 'y' in terms of 't'
    # Output is second derivative of 'y' with respect to 'x' ('d2y/dx2') as a function of 't'
    dy_dx = sp.Derivative(y, t) / sp.Derivative(x, t)
    d2y_dx2 =  sp.Derivative(dy_dx, t) / sp.Derivative(x, t)
    d2y_dx2_in_terms_of_t = sp.nsimplify(d2y_dx2.doit())


    return d2y_dx2_in_terms_of_t
    

In [61]:
c, f_NFW, x_prime = sp.symbols("c f_NFW x_prime", positive=True)

# For NFW AMC

crho_NFW = 1 / (c*x_prime) / (1+c*x_prime)**2 # dimensionless density of NFW profile. 'chro_NFW = rho_NFW / rho_s', where 'rho_NFW' is the density profile and 'rho_s' is the scale radius.
psi_2_NFW = 1/f_NFW * ( sp.log(1+c*x_prime)/x_prime - c/(1+c*x_prime) ) # normalized relative potential. From equation (25) in our article

d2crho_dpsi2_NFW = second_derivative_parametric_input(psi_2_NFW, crho_NFW, x_prime) # Second derivative of 'chro_NFW' with respect to 'psi_2_NFW' as a functin of 'x_prime'


# For Hernquist AMC

crho_Hern = 1 / (c*x_prime) / (1+c*x_prime)**3 # dimensionless density of Hernquist profile. 'chro_Hern = rho_Hern / rho_1', where 'rho_Hern' is the density profile and 'rho_1' is the scale radius.
psi_2_Hern = (1+c)**2 * x_prime / (1+c*x_prime)**2 # normalized relative potential. From equation (30) in our article

d2crho_dpsi2_Hern = second_derivative_parametric_input(psi_2_Hern, crho_Hern, x_prime) # Second derivative of 'chro_Hern' with respect to 'psi_2_Hern' as a functin of 'x_prime'



In [62]:
c_val = 100      #Initializing the concentration of the unperturbed NFW AMC

# NFW functions

# compute normalized crossover radius 'x*'
def get_x_star_NFW():
    eqn = lambda x_c: beta / alpha_squared * E_frac * x_c**2 - ( 1/f_NFW_val * ( np.log(1+c_val*x_c) / x_c  -  c_val / (1+c_val*x_c) ) )
    sol = so.fsolve(eqn, 1)[0]
    
    return sol

# Computes normalized radius 'x' given a value of normalized relative potential 'psi_B', using equation (25) in our article
def get_x_prime_NFW(psi_prime):
    eqn = lambda x_Prime: psi_prime  -  1/f_NFW_val * ( np.log(1+c_val*x_Prime)/x_Prime - c_val/(1+c_val*x_Prime) )
    #sol = so.fsolve(eqn, 1)[0]
    sol = so.root_scalar(eqn, bracket=(x_star, 1e20)).root
    
    return sol

# Compute the integrand according to equations (J8) or (23) in our article
def integrand_1_NFW(psi_prime, eps, x):
    
    psi_1_NFW = 1/f_NFW_val * ( math.log(1+c_val*x)/x - c_val/(1+c_val*x_star) )
    x_prime = get_x_prime_NFW(psi_prime)

    return (
        x**2 * 1/(math.sqrt(8)*math.pi**2) * 1/np.sqrt(eps - psi_prime) * 
        d2crho_dpsi2_c_and_fNFW_substituted_lambdified_NFW(x_prime).real * np.sqrt(2*(psi_1_NFW - eps))
    )


def range_x_1():
    return [0, min(x_star, 1)]

def range_x_1_NFW_I():
    return [0, x_star]

# compute the integrand according to equation (21) in our article
def integrand_2(x):
    return x / (1 + c_val*x)**2

def range_x_2():
    return [0, min(x_star, 1)]


# Hernquist functions

# compute conentration of Hernquist profile given its scale density. Using equation (55) in our article
def compute_c_Hern(rho):
    eqn = lambda c_Hern: 1 / 2 / c_Hern / (1 + c_Hern)**2   -   200 / 3 * rho_crit / rho
    sol = so.fsolve(eqn, 1)[0]
    return sol



# Functions common for both NFW and Hernquist AMCs. Therefore, no suffix.

def range_eps_1(x):
    return [0, beta / alpha_squared * E_frac * x**2]

def range_psi_prime_1(eps, x):
    return [0, eps]



#------------------------------------------------------------------------


if incremental_Efrac == 0.1:
    survival_fractions_multiple_encounters = np.load(r"survival_fractions_Efrac_zeroPoint1_multiple_encounters_fixed_Efrac_switching.npy")
    E_fracs_incremental = np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2])        #incremental Efrac = 0.1
    E_fracs_cumulative = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0])
    p = np.load(f"Value of 'p' for E_frac = 0.1.npy")
elif incremental_Efrac == 7.8:
    survival_fractions_multiple_encounters = np.load(r"survival_fractions_Efrac_7Point8_multiple_encounters_fixed_Efrac_switching.npy")
    E_fracs_incremental = np.array([7.8, 7.8, 7.8, 7.8, 7.8])        #incremental Efrac = 7.8
    E_fracs_cumulative = np.array([7.8, 7.8*2, 7.8*3, 7.8*4, 7.8*5])
    p = np.load(f"Value of 'p' for E_frac = 7.8.npy")
else:
    raise ValueError("Invalid value of incremental_Efrac. Please set 'incremental_Efrac' to one of the following values: 0.1, 7.8")



rho_crit = 9.1275e-27 # in kg/m^3. Cosmological critical density today



f_NFW_val = math.log(1+c_val) - c_val/(1+c_val) # using equation (14) in our article

alpha_squared = 3/c_val**2 + 1/(2*f_NFW_val)*(c_val-3)/(c_val+1)
beta = (c_val**3 - 2*c_val*(1+c_val)*f_NFW_val) / (2*(1+c_val)**2 * f_NFW_val**2) # this is actually gamma in our article

d2crho_dpsi2_c_and_fNFW_substituted_NFW = d2crho_dpsi2_NFW.subs({c:c_val, f_NFW:f_NFW_val}) # substitute the numerical values of 'concentration' and 'f_NFW' in 'd2crho_dpsi2_NFW'
d2crho_dpsi2_c_and_fNFW_substituted_lambdified_NFW = sp.lambdify(x_prime, d2crho_dpsi2_c_and_fNFW_substituted_NFW) # Lambdify 'd2crho_dpsi2_c_and_fNFW_substituted_NFW' so that you can evaluate it by passing a numerical
                                                                                                                   # value of 'x_prime' as input

# Scale densities of AMCs

rho_s = 200/3 * c_val**3/(np.log(1+c_val) - c_val/(1+c_val)) * rho_crit # kg/m^3 . Scale density of of NFW minihalo






effective_Efracs = np.zeros(np.size(E_fracs_incremental))

survival_fractions = np.zeros(np.size(E_fracs_incremental)) # Create a zeros array the same size as the 'E_fracs_incremental' array. This will eventually contain the values of survival fractions corresponding to E_frac values.


for i in range(np.size(E_fracs_incremental)):

    # Effective Efrac
    E_frac = np.sum( E_fracs_incremental[:i+1]**(p/2) )**(2/p)  # considers stellar encounters up to the 'i_th' ecnounter. uses equation (71) in our article
    effective_Efracs[i] = E_frac[0] # 'E_frac' is actually a single element numpy array. We need to extract inside the array by using 'E_frac[0]'

    x_star = get_x_star_NFW() # compute normalized crossover radius x*

# --------------------------------------------------------------------------

    # Method 1

    # Compute mass of 0 < x < x_star region assuming no mass loss in this region - computes the survival fraction equivalent
    mass_fraction = c_val**2/f_NFW_val * si.nquad(integrand_2, [range_x_2])[0]

    # Compute partial mass loss in 0 < x < x_star region - computes the survival fraction equivalent
    partial_mass_loss_fraction = 4*math.pi*c_val**3/f_NFW_val * si.nquad(integrand_1_NFW, [range_psi_prime_1, range_eps_1, range_x_1])[0]

    survival_fraction_1 = mass_fraction - partial_mass_loss_fraction # uses equation (20) in our article

# ------------------------------------------------------------------------

    # Method 2

    # Compute concentration of first Hernquist AMC
    K = np.log(1 + c_val*x_star) - c_val*x_star/(1 + c_val*x_star) # this is 'f_NFW(c_s * x_s*)'
    I, error = si.nquad(integrand_1_NFW, [range_psi_prime_1, range_eps_1, range_x_1_NFW_I]) # from equation (J8) in our article
    R_s = np.sqrt(2*K - 8*np.pi*c_val**3*I) #from equation (45) in our article
    rho_1 = rho_s / R_s # from equation (42) in our article
    c_1 = compute_c_Hern(rho_1) # compute the concentratin of the Hernquist profile given its scale density


    x_1_rVirS = c_val/c_1 / R_s # equation (K3) in our article
    H = (c_1*x_1_rVirS)**2 / (1 + c_1*x_1_rVirS)**2 # this is just 'f_Hern(c_1*x_1_rVirS)'
    survival_fraction_2 = 1/2 * R_s**2 * H/f_NFW_val # equation (K9) in our article

# ---------------------------------------------------------------------------

    survival_fractions[i] = min(survival_fraction_1, survival_fraction_2) # this is the crux of the swtiching procedure

    print(f"Iteration #{i+1}")



Iteration #1
Iteration #2
Iteration #3
Iteration #4
Iteration #5


In [63]:
# Save the value of survival fractions to disk if 'save_to_disk' is set to True
if save_to_disk:
    if incremental_Efrac == 0.1:
        np.save("survival fractions for effective single encounter; incremental E_frac = 0.1", survival_fractions)
    elif incremental_Efrac == 7.8:
        np.save("survival fractions for effective single encounter; incremental E_frac = 7.8", survival_fractions)